### CM4603 – Coursework 2 (Individual)
#### March 2025

Name  : < Pooja Illangarathne > |
RGU ID: < 2117529 >

In [ ]:
from google.colab import drive
import pandas as pd
import os
import json
import re


drive.mount('/content/drive')  # Mounting Google Drive

Mounted at /content/drive


# Answer to Task 4

In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-nmo94uan
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-nmo94uan
  Running command git checkout -q 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Resolved https://github.com/huggingface/transformers to commit 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936429 sha256=5970d791c4bb9795e222f54263c68dd094043d1254261bc6e4a9678039851da9
  Stored in directory: /tmp/pip-ephem-wheel-cache-rlf9dfm0/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transf

In [ ]:
!pip install -U transformers

In [ ]:
import os
import numpy as np
import sentencepiece as spm
from gensim.models import FastText
from sklearn.neighbors import NearestNeighbors
import google.generativeai as genai
from google.colab import userdata

_The implementation integrates FastText embeddings, SentencePiece BPE tokenization, and the Gemini API to enable a retrieval-augmented generation (RAG) system for answering Sinhala-language questions related to the Sri Lankan Constitution. Initially, the Gemini API key is retrieved and configured to interact with Google's language model. The FastText word embeddings and SentencePiece BPE tokenizer are then loaded to handle text processing. The Sri Lankan Constitution is read and stored in a list of passages, with each passage's embedding precomputed using FastText. A nearest neighbor index (cosine similarity) is built to retrieve relevant passages when a user asks a question._

_For context retrieval, the input query is converted into an embedding vector and the top three most similar passages are fetched using the nearest neighbors algorithm. The Gemini model is then used in two modes: (1) Answering questions without RAG, where the model directly generates a response based on the question alone, and (2) Answering with RAG, where the retrieved passages are appended to the query to provide more contextually relevant answers._

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=300):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.0
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය ඇත, නමුත් එම බලය සීමා සහිතය.  ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබාදිය හැකිය.  කෙසේ වුවද, මෙම බලය සම්පූර්ණයෙන්ම අසීමිත නොවේ.  

විශේෂයෙන්ම, ජනාධිපතිවරයාට පහත සඳහන් අවස්ථාවලදී සමාව ලබාදිය නොහැක:

* **මහජන නියෝජිතයින්ගේ හෝ රාජ්‍ය නිලධාරීන්ගේ

Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: **සිංහල** සහ **தமிழ் (தமிழ்)**.


=== Answers WITH RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
Retrieved Context: (2) පාර්ලිමේන්තුව විසින් ආණ්ඩුවේ ප්‍රතිපත්ති ප්‍රකාශය හෝ විසර්ජන පනත් කෙටුම්පත හෝ ප්‍රතික්ෂේප කළහොත් එ

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=300):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.0
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය ඇත, නමුත් එම බලය සීමා සහිතය.  ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබාදිය හැකිය.  කෙසේ වුවද, මෙම බලය සම්පූර්ණයෙන්ම අසීමිත නොවේ.  

විශේෂයෙන්ම, ජනාධිපතිවරයාට පහත සඳහන් අවස්ථාවලදී සමාව ලබාදිය නොහැක:

* **මහජන නියෝජිතයින්ගේ හෝ රාජ්‍ය නිලධාරීන්ගේ

Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: **සිංහල** සහ **தமிழ் (தமிழ்)**.


Q: ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?
A: ඕස්ට්‍රේලියාවේ අගනුවර **කැන්බරා** ය.


Q: ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?
A: ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ හතර ශ්‍රී ලංකාවේ ඓතිහාසික රාජධානි හතර නිය

Performance Comparison: With vs. Without RAG

**Responses Without RAG**

Gemini generates factually correct answers most of the time without referring to the actual constitution.
Answers are concise, well-structured, and fluent in Sinhala.

Potential Issues:

Some answers lack legal references or specific constitutional articles.
Answers may be based on general knowledge rather than the actual Sri Lankan Constitution.
Partial answers (e.g., the pardon question was incomplete).

Example:
Question: "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?"

Answer Without RAG:
"ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: සිංහල සහ தமிழ் (தமிழ்)."

Correct but lacks a specific legal reference to a constitutional article.

**Responses With RAG (Retrieval-Augmented Generation)**

The chatbot retrieves relevant context from the constitution before answering.
Legal references (constitutional clauses) are included in responses.
More trustworthy and verifiable answers.

Potential Issues:

Sometimes irrelevant or too much text is retrieved.
Some answers still cut off or remain incomplete (possible Gemini truncation issue).

Example:
Question: "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?"
Retrieved Context: Mentions governance, official languages, and legislative authority.

Answer With RAG:

"ශ්‍රී ලංකා ආණ්ඩුක්‍රම ව්‍යවස්ථාවේ ඒ ගැන සෘජුවම සඳහන් වන්නේ නැත. කෙසේ වුවද, ලබාදී ඇති උපුටා ගැනීම්වලින් හෝ ශ්‍රී ලංකා ආණ්ඩුක්‍රම ව්‍යවස්ථාවේ වෙනත් වගන්තිවලින් හෝ ජාතික භාෂා ලෙස සිංහල හා දෙමළ භාෂා දෙකම හඳුනාගත හැකිය."

More accurate than the non-RAG answer, but vague about the exact constitutional article.

Strengths of the Chatbot

Fluent Sinhala Generation: Gemini provides highly fluent and natural Sinhala responses.

Handles Constitutional Queries Well: Even without RAG, Gemini understands legal concepts and provides correct responses.

RAG Improves Credibility: The retrieved context from the constitution improves factual accuracy.

FastText Enhances Query Understanding: The FastText embeddings with BPE tokenization help handle morphologically complex Sinhala words, improving query retrieval accuracy.

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=300):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.1
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය ඇත, නමුත් එම බලය සීමා සහිතය.  ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබා දිය හැකිය.  කෙසේ වුවද, මෙම බලය සම්පූර්ණයෙන්ම අසීමිත නොවේ.  

විශේෂයෙන්ම, ජනාධිපතිවරයාට පාර්ලිමේන්තුව විසින් චෝදනා ගොනු කරන ලද වරදකට සමාව ලබා දිය නොහැක.  එසේම, යම් වරදක් ස

Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: **සිංහල** සහ **தமிழ் (தமிழ்)**.


Q: ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?
A: ඕස්ට්‍රේලියාවේ අගනුවර **කැන්බරා** ය.


Q: ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?
A: ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ හතර ශ්‍රී ලංකාවේ ඓතිහාසික රාජධානි හතර නියෝ

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=500):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.2
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය ඇත, නමුත් එම බලය සීමා සහිතය.  ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබාදිය හැකිය.  කෙසේ වුවද, මෙම බලය සම්පූර්ණයෙන්ම නිරපේක්ෂ නොවේ.  

විශේෂයෙන්ම, ජනාධිපතිවරයාට පහත සඳහන් අවස්ථාවලදී සමාව ලබාදිය නොහැක:

* **මහජන නියෝජිතයින්ගේ හෝ රාජ්‍ය නිලධාරීන්ගේ වංචා චෝදනා:**  මෙම චෝදනා සම්බන්ධයෙන් ජනාධිපතිවරයාට සමාව ලබාදිය හැක්කේ පාර්ලිමේන්තුවේ බහුතරයක අනුමැතිය ඇතිව පමණි.  මෙය වංචා චෝදනාවල බරපතලකම සැලකිල්ලට ගනිමිනි.

* **ව්‍යවස්ථාවෙන් නිශ්චිතවම තහනම් කර ඇති අපරාධ:**  ව්‍යවස්ථාවෙන්ම සමාව 

Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: **සිංහල** සහ **தமி

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=700):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.2
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව, ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය ඇත, නමුත් එම බලය සීමා සහිතය.  ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබාදිය හැකිය.  කෙසේ වුවද, මෙම බලය සම්පූර්ණයෙන්ම අසීමිත නොවේ.  

විශේෂයෙන්ම, ජනාධිපතිවරයාට පහත සඳහන් අවස්ථාවලදී සමාව ලබාදිය නොහැක:

* **මහජන නියෝජිතයින්ගේ හෝ රාජ්‍ය නිලධාරීන්ගේ වංචා චෝදනා:**  මෙම චෝදනා සම්බන්ධයෙන් ජනාධිපතිවරයාට සමාව ලබාදිය හැක්කේ පාර්ලිමේන්තුවේ තුනෙන් දෙකක බහුතර ඡන්දයකින් පසුව පමණි.
* **ව්‍යවස්ථාව උල්ලංඝනය කිරීම:** ව්‍යවස්ථාවේ පැහැදිලිව සඳහන් කර ඇති උල්ලංඝනයන් සඳහා ජනාධිපතිවරයාට සමාව ලබාදිය නොහැක.

එබැවින්, ජනාධිපතිවරයාට සමාව ලබාදීමේ බලය තිබුණද, එම බලය අසීමිත නොවන අතර, ව්‍යවස්ථාවේ සීමාවන්ට යටත් වේ.  විශේෂිත චෝදනාවක් 

In [ ]:
# Retrieve and configure the Gemini API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in Colab secrets.")
genai.configure(api_key=GOOGLE_API_KEY)

# Loading FastText and SentencePiece models
FASTTEXT_MODEL_PATH = "fasttext_sinhala_bpe.model"
BPE_MODEL_PATH = "sinhala_spm_bpe.model"
CONSTITUTION_PATH = "/content/drive/MyDrive/Sri Lanka Constitution-Sinhala.txt"

ft_model = FastText.load(FASTTEXT_MODEL_PATH)
sp = spm.SentencePieceProcessor(model_file=BPE_MODEL_PATH)

# Loading Constitution passages
with open(CONSTITUTION_PATH, "r", encoding="utf-8") as f:
    passages = [line.strip() for line in f if line.strip()]

# Computing averaged FastText embedding using BPE tokens
def get_embedding(text):
    tokens = sp.encode(text, out_type=str)
    valid_vectors = [ft_model.wv[t] for t in tokens if t in ft_model.wv.key_to_index]
    return np.mean(valid_vectors, axis=0) if valid_vectors else np.zeros(ft_model.vector_size)

# Precomputing passage embeddings and build a nearest neighbor index (cosine similarity)
passage_embeddings = np.array([get_embedding(p) for p in passages])
nn = NearestNeighbors(n_neighbors=3, metric="cosine")  # Retrieving top 3 passages
nn.fit(passage_embeddings)

# Retrieving context from the Constitution based on the question
def retrieve_context(question):
    query_emb = get_embedding(question).reshape(1, -1)
    distances, indices = nn.kneighbors(query_emb)
    return " ".join([passages[i] for i in indices[0]])

# Initializing the Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate an answer using the Gemini API
def generate_answer(prompt, max_tokens=700):
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=0.3
        )
    )
    return response.text

# Answer without RAG: Direct generation without context
def answer_without_rag(question):
    prompt = f"Q: {question}\nA:"
    return generate_answer(prompt)

# Answer with RAG: Combine retrieved context with the question
def answer_with_rag(question):
    context = retrieve_context(question)
    print("Retrieved Context:", context)
    prompt = (
        f"Instructions: Use the provided context to answer the question accurately and completely in Sinhala. "
        f"Include specific references to articles or sections of the Sri Lankan Constitution when relevant. "
        f"If the context is insufficient or irrelevant, use your general knowledge to provide a correct answer.\n"
        f"Context: {context}\n"
        f"Q: {question}\n"
        f"A:"
    )
    return generate_answer(prompt)

# Sample questions for testing
sample_questions = [
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?",
    "ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?",
    "ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?",
    "ශ්‍රී ලංකා ජාතික කොඩියේ ඇති බෝ කොළ 4න් අදහස් කරන්නේ කුමක්ද?",
    "Is Tokyo the capital city of Japan ?",
    "What are the powers of the President of Sri Lanka?",
    "ශ්‍රී ලංකාවේ ජනාධිපතිවරයාගේ බලතල මොනවාද?"
]

# Test both approaches
print("=== Answers WITHOUT RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_without_rag(q))
    print("====================================\n")

print("=== Answers WITH RAG ===")
for q in sample_questions:
    print("Q:", q)
    print("A:", answer_with_rag(q))
    print("====================================\n")

=== Answers WITHOUT RAG ===
Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අගමැතිද?
A: ඔව්, ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව අමාත්‍ය මණ්ඩලයට ප්‍රධානයා වන්නේ අග්‍රාමාත්‍යවරයාය.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට යම් වරදක් වෙනුවෙන් සම්පෂූර්ණ සමාව ලබා දීමේ බලතල තිබේද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලතල ඇත, නමුත් එම බලතල සීමිතයි.  1978 ව්‍යවස්ථාවේ 34(1) වගන්තියට අනුව, ජනාධිපතිවරයාට ඕනෑම චෝදනාවකට හෝ වරදකට සමාව ලබා දිය හැකිය.  කෙසේ වුවද, මෙම බලය යම් සීමාවන්ට යටත් වේ.  උදාහරණයක් ලෙස, ජනාධිපතිවරයාට රාජද්‍රෝහී ක්‍රියාවකට හෝ රාජ්‍ය භාරකාරත්වය උල්ලංඝනය කිරීමකට සමාව ලබා දිය නොහැක.  එසේම, ජනාධිපතිවරයාට ලබා දිය හැකි සමාවේ පරිමාණය සහ එහි වලංගුභාවය උසාවිවලින් පරීක්ෂා කළ හැකිය.  එබැවින්, ජනාධිපතිවරයාට සම්පූර්ණ සමාව ලබාදීමේ බලය තිබුණද, එය අසීමිත බලයක් නොවේ.


Q: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව මෙරට ජාතික භාෂා මොනවාද?
A: ශ්‍රී ලංකා ව්‍යවස්ථාව අනුව රටේ ජාතික භාෂා දෙකක් තිබේ: **සිංහල** සහ **தமிழ் (தமிழ்)**.


Q: ඕස්ට්‍රේලියාවේ අගනුවර කුමක්ද?
A: ඕස්ට්‍රේලිය